In [46]:
import numpy as np
import random
import math

In [47]:
initial_state = []
action_space = ['hit','stick']
value_dict, total_value, total_visits = {},{},{}
for player_sum in range(2,22):
    for dealer_card in range(2,12):
            initial_state.append([player_sum,dealer_card])
            for action in action_space:
                value_dict[player_sum,dealer_card,action] = 0
                total_value[player_sum,dealer_card,action] = 0
                total_visits[player_sum,dealer_card,action] = 0

In [48]:
def draw_card():
    card_pool = [i for i in range(2,12)] + [10, 10, 10]
    return np.random.choice(card_pool)

def dealer_operation(dealer_value):
    if dealer_value > 21:
        return 'blow up'
    if dealer_value >= 18:
        return dealer_value
    if dealer_value < 18:
        return dealer_operation(dealer_value + draw_card())

def my_strategy(player_value,dealer_value):
    global best_action
    if player_value > 21:
        return 'blow up'
    if total_visits[player_value,dealer_value,'hit'] == 0:
        best_action = 'hit'
    elif total_visits[player_value,dealer_value,'stick'] == 0:
        best_action = 'stick'
    elif total_visits[player_value,dealer_value,'hit'] != 0 and total_visits[player_value,dealer_value,'stick'] != 0:
        state_visits = total_visits[player_value,dealer_value,'hit'] + total_visits[player_value,dealer_value,'stick'] + 1
        candidate_hit = value_dict[player_value,dealer_value,'hit'] + c*math.sqrt(math.log(state_visits) / total_visits[player_value,dealer_value,'hit'])
        candidate_stick = value_dict[player_value,dealer_value,'stick'] + c*math.sqrt(math.log(state_visits) / total_visits[player_value,dealer_value,'stick'])
        if candidate_stick > candidate_hit:
            best_action = 'stick'
        elif candidate_stick <= candidate_hit:
            best_action = 'hit'
    if best_action == 'hit':
        visited_states_action_pairs.append([player_value,dealer_value,'hit'])
        return my_strategy(player_value + draw_card(),dealer_value)
    elif best_action == 'stick':
        visited_states_action_pairs.append([player_value,dealer_value,'stick'])
        return player_value

def winner(player_value,dealer_value):
    global result
    if player_value == 'blow up':
        return -1
    if dealer_value == 'blow up':
        return 1
    if player_value > dealer_value:
        return 1
    if player_value == dealer_value:
        return 0
    if player_value < dealer_value:
        return -1

In [49]:
c = 1
for i in range(100000):
    visited_states_action_pairs = []
    start = random.choice(initial_state)
    player_final = my_strategy(start[0],start[1])
    episode_result = winner(player_final, dealer_operation(start[1]))
    #print(visited_states_action_pairs)
    for i in visited_states_action_pairs:
        total_value[i[0],i[1],i[2]] += episode_result
        total_visits[i[0],i[1],i[2]] += 1
        value_dict[i[0],i[1],i[2]] = round(total_value[i[0],i[1],i[2]] / total_visits[i[0],i[1],i[2]], 3)

print(total_visits)
print(value_dict)
print(my_strategy(2,3))

{(2, 2, 'hit'): 291, (2, 2, 'stick'): 224, (2, 3, 'hit'): 364, (2, 3, 'stick'): 157, (2, 4, 'hit'): 513, (2, 4, 'stick'): 18, (2, 5, 'hit'): 15, (2, 5, 'stick'): 468, (2, 6, 'hit'): 363, (2, 6, 'stick'): 84, (2, 7, 'hit'): 308, (2, 7, 'stick'): 194, (2, 8, 'hit'): 389, (2, 8, 'stick'): 130, (2, 9, 'hit'): 447, (2, 9, 'stick'): 48, (2, 10, 'hit'): 428, (2, 10, 'stick'): 119, (2, 11, 'hit'): 315, (2, 11, 'stick'): 202, (3, 2, 'hit'): 407, (3, 2, 'stick'): 68, (3, 3, 'hit'): 314, (3, 3, 'stick'): 184, (3, 4, 'hit'): 286, (3, 4, 'stick'): 226, (3, 5, 'hit'): 464, (3, 5, 'stick'): 58, (3, 6, 'hit'): 318, (3, 6, 'stick'): 173, (3, 7, 'hit'): 225, (3, 7, 'stick'): 300, (3, 8, 'hit'): 453, (3, 8, 'stick'): 60, (3, 9, 'hit'): 363, (3, 9, 'stick'): 165, (3, 10, 'hit'): 444, (3, 10, 'stick'): 44, (3, 11, 'hit'): 343, (3, 11, 'stick'): 162, (4, 2, 'hit'): 133, (4, 2, 'stick'): 371, (4, 3, 'hit'): 502, (4, 3, 'stick'): 36, (4, 4, 'hit'): 559, (4, 4, 'stick'): 5, (4, 5, 'hit'): 387, (4, 5, 'stick'):

In [50]:
policy_dict,unreasonable_stick_sum, unreasonable_hit_sum = {},0,0
for player_sum in range(2,22):
    for dealer_card in range(2,12):
        if value_dict[player_sum,dealer_card,'hit'] > value_dict[player_sum,dealer_card,'stick']:
            policy_dict[player_sum,dealer_card] = 'hit'
            if player_sum >= 16:
                unreasonable_hit_sum += 1
        else:
            policy_dict[player_sum,dealer_card] = 'stick'
            if player_sum <= 10:
                unreasonable_stick_sum += 1

print(policy_dict)
print('Number of unreasonable hits at high player value: ',unreasonable_hit_sum)
print('Number of unreasonable sticks at low player value: ',unreasonable_stick_sum)



{(2, 2): 'hit', (2, 3): 'hit', (2, 4): 'hit', (2, 5): 'stick', (2, 6): 'hit', (2, 7): 'hit', (2, 8): 'hit', (2, 9): 'hit', (2, 10): 'hit', (2, 11): 'hit', (3, 2): 'hit', (3, 3): 'hit', (3, 4): 'hit', (3, 5): 'hit', (3, 6): 'hit', (3, 7): 'hit', (3, 8): 'hit', (3, 9): 'hit', (3, 10): 'hit', (3, 11): 'hit', (4, 2): 'hit', (4, 3): 'hit', (4, 4): 'hit', (4, 5): 'hit', (4, 6): 'stick', (4, 7): 'stick', (4, 8): 'hit', (4, 9): 'hit', (4, 10): 'stick', (4, 11): 'hit', (5, 2): 'hit', (5, 3): 'hit', (5, 4): 'stick', (5, 5): 'hit', (5, 6): 'hit', (5, 7): 'stick', (5, 8): 'hit', (5, 9): 'hit', (5, 10): 'stick', (5, 11): 'hit', (6, 2): 'stick', (6, 3): 'hit', (6, 4): 'hit', (6, 5): 'stick', (6, 6): 'hit', (6, 7): 'stick', (6, 8): 'hit', (6, 9): 'hit', (6, 10): 'hit', (6, 11): 'hit', (7, 2): 'hit', (7, 3): 'hit', (7, 4): 'hit', (7, 5): 'stick', (7, 6): 'hit', (7, 7): 'hit', (7, 8): 'hit', (7, 9): 'hit', (7, 10): 'stick', (7, 11): 'hit', (8, 2): 'hit', (8, 3): 'hit', (8, 4): 'hit', (8, 5): 'hit', (8,